<a href="https://colab.research.google.com/github/ejrtks1020/Pytorch-basic/blob/main/pytorch-tutorials/pytorch_neural_network_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 신경망 모델구성하기
## reference : https://tutorials.pytorch.kr/beginner/basics/buildmodel_tutorial.html

In [1]:
import os
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

In [4]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f'using {device} device')

using cuda device


In [5]:
# 클래스 정의하시
# 신경망 모델을 nn.Module의 하위클래스로 정의
# __init__에서 신경망 계츨들을 초기화
# nn.Module을 상속받은 모든 클래스는 forward 메소드에 입력데이터에 대한 연산을 구현

class NeuralNetwork(nn.Module):
  def __init__(self):
    super(NeuralNetwork, self).__init__()
    self.flatten = nn.Flatten()
    self.linear_relu_stack = nn.Sequential(
        nn.Linear(28 * 28, 512),
        nn.ReLU(),
        nn.Linear(512, 512),
        nn.ReLU(),
        nn.Linear(512, 10)
    )
  def forward(self, x):
    x = self.flatten(x)
    logits = self.linear_relu_stack(x)
    return logits

In [6]:
# NeuralNetwork 인스턴스 생성하고 device로 이동
model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [10]:
X = torch.rand(1, 28, 28, device = device)

# 모델에 랜덤 입력값을 호출하면 10차원 raw prediction 텐서반환
logits = model(X)
print(logits)

# 원시 예측값(logits)을 Softmax 모듈의 인스턴스에 통과시켜 예측확률 반환
pred_probab = nn.Softmax(dim=1)(logits)
print(pred_probab)
y_pred = pred_probab.argmax(dim = 1)
print(f"Predicted class: {y_pred.item()}")

tensor([[-0.0491,  0.0114, -0.0243, -0.0610, -0.0812,  0.0540,  0.0124, -0.0847,
         -0.0999,  0.0175]], device='cuda:0', grad_fn=<AddmmBackward0>)
tensor([[0.0980, 0.1041, 0.1005, 0.0969, 0.0949, 0.1087, 0.1043, 0.0946, 0.0932,
         0.1048]], device='cuda:0', grad_fn=<SoftmaxBackward0>)
Predicted class: 5


# 모델 계층(Layer)

In [12]:
input_image = torch.rand(3, 28, 28)
print(input_image.size())

flatten = nn.Flatten()
# Flatten시 dim = 0 미니배치 차원은 유지
flat_image = flatten(input_image)
print(flat_image.size())

torch.Size([3, 28, 28])
torch.Size([3, 784])


In [13]:
# nn.Linear : 저장된 가중치와 편향을 사용하여 입력에 선형변환을 적용하는 모듈
layer1 = nn.Linear(in_features = 28 * 28, out_features = 20)
hidden1 = layer1(flat_image)
print(hidden1.size())

torch.Size([3, 20])


In [14]:
# nn.ReLU : 활성화함수, 선형 변환후 적용되어 비선형성(non-linearity)를 적용하고 신경망에 다양한 현상을 학습할 수 있도록 돕는다.
print(f"Before ReLU: {hidden1}\n\n")
hidden1 = nn.ReLU()(hidden1)
print(f"After ReLU: {hidden1}")

Before ReLU: tensor([[ 0.1532, -0.0111, -0.2498,  0.7671,  0.1364,  0.4962, -0.4281,  0.1205,
          0.3608, -0.2032, -0.0313, -0.5315, -0.2074, -0.0844, -0.3556,  0.4682,
         -0.0279,  0.3472, -0.1486,  0.3224],
        [ 0.1001, -0.0518, -0.0139,  0.5452,  0.3738,  0.2689, -0.0248,  0.3211,
          0.3112, -0.1966,  0.2742, -0.7231,  0.2708, -0.6037, -0.4260,  0.3563,
         -0.0530,  0.2326, -0.1163,  0.1091],
        [ 0.1380, -0.0049,  0.0837,  0.6237,  0.5123,  0.5750, -0.2520, -0.0984,
          0.0658, -0.2908,  0.1795, -0.7886, -0.0260, -0.2406, -0.2490,  0.5834,
          0.0625,  0.8358, -0.3057,  0.2657]], grad_fn=<AddmmBackward0>)


After ReLU: tensor([[0.1532, 0.0000, 0.0000, 0.7671, 0.1364, 0.4962, 0.0000, 0.1205, 0.3608,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.4682, 0.0000, 0.3472,
         0.0000, 0.3224],
        [0.1001, 0.0000, 0.0000, 0.5452, 0.3738, 0.2689, 0.0000, 0.3211, 0.3112,
         0.0000, 0.2742, 0.0000, 0.2708, 0.0000, 0.00

In [15]:
# nn.Sequential : 순서를 갖는 모듈의 컨테이너 (sequential container)

seq_modules = nn.Sequential(
    flatten,
    layer1,
    nn.ReLU(),
    nn.Linear(20, 10)
)
input_image = torch.rand(3, 28, 28)
logits = seq_modules(input_image)

In [16]:
print(logits)

tensor([[ 0.0554,  0.2739, -0.1428,  0.1716,  0.2131,  0.1590,  0.1146,  0.1779,
          0.2681,  0.0726],
        [ 0.0557,  0.1059, -0.1052,  0.2343,  0.2804,  0.1909,  0.0311,  0.2163,
          0.2482,  0.1422],
        [ 0.0416,  0.2807, -0.1624,  0.1130,  0.1420,  0.0678,  0.0927,  0.1470,
          0.2756,  0.1629]], grad_fn=<AddmmBackward0>)


In [17]:
# nn.Softmax : 신경망의 마지막 레이어는 nn.Softmax 모듈에 전달될 [-infty, infty]범위의 raw value인 logits를 반환
# logits는 Softmax모듈에 의해 모델의 각 클래스에 대한 예측확률을 나타내도록 [0, 1]범위로 비례하여 조정됨(scale)
# dim 매개변수는 값의 합이 1이되는 차원을 말함

softmax = nn.Softmax(dim = 1)
pred_probab = softmax(logits)
print(pred_probab)

tensor([[0.0916, 0.1140, 0.0752, 0.1029, 0.1073, 0.1016, 0.0972, 0.1036, 0.1134,
         0.0932],
        [0.0913, 0.0960, 0.0778, 0.1092, 0.1144, 0.1046, 0.0891, 0.1073, 0.1107,
         0.0996],
        [0.0922, 0.1171, 0.0752, 0.0990, 0.1019, 0.0946, 0.0970, 0.1024, 0.1165,
         0.1041]], grad_fn=<SoftmaxBackward0>)


In [20]:
#모델 Prameter(매개변수)
# 각 파라미터들을 순회하며(iterate), 매개변수의 크기와 값을 출력

print("Model structure: ", model, "\n\n")

for name, param in model.named_parameters():
  print(f"Layer: {name} | Size: {param.size()} | Value: {param[:2]} \n")

Model structure:  NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
) 


Layer: linear_relu_stack.0.weight | Size: torch.Size([512, 784]) | Value: tensor([[-3.4527e-02,  3.3292e-02,  8.8656e-03,  ...,  1.1432e-03,
          1.5960e-02, -1.3422e-02],
        [ 1.7254e-02, -2.1265e-02, -2.7931e-03,  ..., -3.5461e-03,
         -6.4416e-05,  1.9675e-02]], device='cuda:0', grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.0.bias | Size: torch.Size([512]) | Value: tensor([ 0.0202, -0.0284], device='cuda:0', grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.2.weight | Size: torch.Size([512, 512]) | Value: tensor([[-0.0295,  0.0068,  0.0122,  ...,  0.0222, -0.0015,  0.0380],
        [-0.0035,  0.0109, -0.0169,  ..., -0.0113, 